In [5]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("from_hive") \
    .master("spark://spark-master:7077") \
    .config("spark.sql.catalogImplementation", "hive") \
    .config("hive.metastore.uris", "thrift://hive-metastore:9083") \
    .enableHiveSupport() \
    .getOrCreate()

In [6]:
df_load = spark.sql('show databases')
df_load.show()

+---------+
|namespace|
+---------+
|  default|
+---------+



In [7]:
spark.stop()

Попробуем создать из этого файла несколько паркетников

In [8]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("csv_to_many_parquets") \
    .master("spark://spark-master:7077") \
    .config("spark.sql.catalogImplementation", "hive") \
    .config("hive.metastore.uris", "thrift://hive-metastore:9083") \
    .enableHiveSupport() \
    .getOrCreate()

from pyspark.sql.types import StructType, StructField, IntegerType, StringType
schema = StructType([
    StructField("NUM", IntegerType(), True),
    StructField("NAME", StringType(), True),
    StructField("CITY", StringType(), True),
    StructField("STATE", StringType(), True),
    StructField("ID", IntegerType(), True),
])

df = spark.read.option("header", "false") \
    .schema(schema) \
    .csv("hdfs://namenode:9000/data/openbeer/breweries/breweries.csv")

df = df.repartition(4)
df.write.mode("overwrite").parquet("hdfs://namenode:9000/data/openbeer/breweries_parquet")

Теперь создадим таблицу в hive

In [9]:
spark.sql("""
CREATE EXTERNAL TABLE IF NOT EXISTS breweries_parquet(
    NUM INT,
    NAME STRING,
    CITY STRING,
    STATE STRING,
    ID INT
)
STORED AS PARQUET
LOCATION 'hdfs://namenode:9000/data/openbeer/breweries_parquet'
""")

25/10/15 15:54:16 WARN SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.


DataFrame[]

In [13]:
spark.sql("SHOW TABLES").show()

+---------+-----------------+-----------+
|namespace|        tableName|isTemporary|
+---------+-----------------+-----------+
|  default|breweries_parquet|      false|
+---------+-----------------+-----------+



In [14]:
spark.sql("SELECT * FROM default.breweries_parquet").show()

+---+--------------------+------------------+-----+---+
|NUM|                NAME|              CITY|STATE| ID|
+---+--------------------+------------------+-----+---+
| 77|    Blue Owl Brewing|            Austin|   TX| 77|
|515|Crabtree Brewing ...|           Greeley|   CO|515|
|222|Carton Brewing Co...|Atlantic Highlands|   NJ|222|
|512|    Cottrell Brewing|         Pawcatuck|   CT|512|
|332|La Cumbre Brewing...|       Albuquerque|   NM|332|
|425|           Ciderboys|     Stevens Point|   WI|425|
|483|Bale Breaker Brew...|            Yakima|   WA|483|
|291|Tommyknocker Brewery|     Idaho Springs|   CO|291|
|350|Central Coast Bre...|   San Luis Obispo|   CA|350|
|334|The Traveler Beer...|        Burlington|   VT|334|
|327|Half Acre Beer Co...|           Chicago|   IL|327|
|299|Matt Brewing Company|             Utica|   NY|299|
|311|Dirty Bucket Brew...|       Woodinville|   WA|311|
|479|     Heavy Seas Beer|        Halethorpe|   MD|479|
|528|Asheville Brewing...|         Asheville|   

In [15]:
spark.stop()